## Simulation Environments (last updated 21.03.31)
- CU with local dataset
- **Local dataset**: Unbalanced (3 classes/local dataset), Non-i.i.d., non-overapped samples
- **Channel**: Pathloss + Rayleigh fading
- **Power Allocation**: Waterfilling Power Allocation
- **Transmission type**: Analog Transmission with Orthogonal sub-channel allocation
- **Fixed device deployment** and **Fixed local dataset**

- CU and deivces have the same weight for the model update

In [1]:
import copy
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import math
import random

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.fc1 = nn.Linear(64 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        #x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(torch.tanh(self.conv1(x)))
        #x = self.pool(F.relu(self.conv2(x)))        
        x = self.pool(torch.tanh(self.conv2(x)))
        x = x.view(-1, 64 * 4 * 4)
        #x = F.relu(self.fc1(x))
        x = torch.tanh(self.fc1(x))
        x = self.fc2(x)
        return x

net = Net()
net.to(device)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
)

### Default Setup *(Do not modify)*

In [3]:
lr = 0.1   # 0.1
batch_local = 10   # Size of local minibatch
E = 1 #3  # Local training epoch
epoch_max = 300
D = sum( p.numel() for p in net.parameters() if p.requires_grad)   # Number of parameters in model, 582,026 

noise_pow_dBm = -50 #-100
P_dBm = 20  #20
gamma = 0.1  # Target received SNR

B = 1024    # Number of subcarrier

cell_radius = 400   # Distance to the farthest devices
density = 250/(np.pi*500**2)   # 200 devices within 500m in average
pathloss_exponent = 4

num_classes = 3   # Number of classes in local dataset
samples_avg_local = 200   # Average number of local samples
samples_CU = samples_avg_local   # Average number of samples at CU

regul_coeff = 10e-5   # L2 regularization parameter for avoiding overfitting
#init_E = E   # Pre-training epoch

### Variations in Setup *(Adjustable)*

In [4]:
P_dBm = 100

### Load Simulation Setups

In [5]:
PATH_DEPLOYMENT = './simulation_setups/Deployment_within_550.npy'
PATH_TRAIN_DATASET = './simulation_setups/train_dataset.pth'
PATH_LOCAL_DATASET = './simulation_setups/localdataset_within_550.npy'

dev_deployment = np.load(PATH_DEPLOYMENT)        # Deployment of devices within 550 meters
trainset = torch.load(PATH_TRAIN_DATASET)         # Load training data and labels
x, y = trainset.values()
idx_localDB = np.load(PATH_LOCAL_DATASET, allow_pickle=True)        # Classified data indices

### Subordinate System Parameters *(Do not modify)*

In [6]:
noise_pow = 10**(noise_pow_dBm/10)
P = 10**(P_dBm/10)
N = int(np.ceil(D/B))   # Number of transmit symbols
dev_num = np.sum((np.linalg.norm(dev_deployment, axis=1)<cell_radius))   # Number of devices
samples_local = [ len(idx_localDB[i]) for i in range(dev_num)]     # Number of samples at devices
samples_global = samples_CU+np.sum(samples_local)
print('Number of participating devices: ', dev_num)
print('Number of utilized samples:', samples_global)

dev_dist = np.linalg.norm(dev_deployment[:dev_num], axis=1)
pathloss = dev_dist**-pathloss_exponent
print('Link distances: ', dev_dist[0:5])
print('Pathloss: ', pathloss[0:5])

permute_idx = np.random.permutation(B*N)
permute_idx_inv = np.argsort(permute_idx)

Number of participating devices:  169
Number of utilized samples: 33989
Link distances:  [22.83855181 34.12266929 41.28552959 42.75910894 59.44096671]
Pathloss:  [3.67557892e-06 7.37612065e-07 3.44198173e-07 2.99147525e-07
 8.01044318e-08]


In [7]:
########### Data classification ##############
idx = []   # Empty list
for i in range(10):
    tmp = list((y==i).nonzero()[0])
    idx.append(tmp)
    
############ CU dataset ############
CU_x = torch.zeros(samples_CU,1, 28, 28)
CU_y = np.zeros(samples_CU, dtype=int)
CU_dataset = []
for i in range(10):
    CU_dataset += idx[i][:int(samples_CU/10)]
random.shuffle(CU_dataset)
    
for i, idx_CU in enumerate(CU_dataset):
    CU_x[i], CU_y[i] = x[idx_CU], y[idx_CU]

################### Test dataset ##################
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
testset = torchvision.datasets.MNIST(root='../shared/MNIST_dataset/', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=10, shuffle=False, num_workers=3)

classes = (0, 1, 2, 3, 4, 5, 6, 7, 8, 9)

In [8]:
def power_allocation(gamma, noise_pow, P, N, B, delta, h_pow, rho, samples_local, samples_global):
    delta_pow_2d = (delta**2).reshape((N, B))   # (N, B)
            
    g = (1/h_pow).reshape((N,B))    
    w = samples_local/samples_global
    u = (gamma*noise_pow/rho) * g * w**2   # (N, B)
    c_init = np.max(delta**2)/(np.min(u))
            
    v = 1.0*delta_pow_2d   #(N,B)
    WF_idx = np.nonzero(1.0*(np.sum(u*delta_pow_2d, axis=1) > P))[0]   # (N, ) Sub-carriers for waterfilling 
            
    for idx in WF_idx:
        c_min = 0
        while np.sum(u[idx]*np.maximum(delta_pow_2d[idx]-c_init*u[idx],0))>P:
            c_init *=2
        c_max = c_init
                                    
        while (c_max - c_min)>c_init*10**-10:
            c_mid = (c_min+c_max)/2
            v_mid = np.maximum(delta_pow_2d[idx]-c_mid*u[idx], 0)
            if np.sum(v_mid*u[idx])>P:
                c_min = c_mid
            else:
                c_max = c_mid
        v[idx] = v_mid
                
    P_alloc = (u*v).reshape(-1)   # (N*B, )

    return P_alloc 

In [9]:
def model_localupdate(inputs, labels, net_org, E, batch_local, samples_local, lr):
    criterion = nn.CrossEntropyLoss()
    running_loss_local = 0.0
    acc_local = 0.0
    net_local = copy.deepcopy(net_org)   # deep copy of the network
    for _ in range(E):   # Local training epoch
        for b in range(int(np.ceil(samples_local/batch_local))):
            
            L2_reg = torch.tensor(0., requires_grad=True).to(device)
            for param in net_local.parameters():
                L2_reg = L2_reg + torch.norm(param, 2)
            
            start_idx = b*batch_local
            end_idx = b*batch_local+np.min([batch_local, samples_local-b*batch_local])
            outputs = net_local(inputs[start_idx:end_idx])
            loss = criterion(outputs, labels[start_idx:end_idx]) + regul_coeff*L2_reg   # 10e-4 L2-Regularization to prevent overfitting 
            net_local.zero_grad()
            loss.backward()
            
            if math.isnan(loss.item()): # Catch error
                print("outputs", outputs)
                print("inputs", inputs[start_idx:end_idx])
                w_sum = 0
                for i in net_local.parameters():
                    w_sum+=torch.sum(i)
                print('Sum of parameters: ', w_sum.item())
                ErrorOccur = input("Nan appears")
            
            with torch.no_grad():
                running_loss_local += loss.item()/(samples_local/batch_local)/E
                _, predicted = torch.max(outputs.data, 1)
                acc_local += (predicted == labels[start_idx:end_idx]).sum().item()
                for param_local in net_local.parameters():
                    param_local -= lr*param_local.grad
                    if math.isnan(torch.sum(param_local).item()):   # Catch error
                        print(param_local)
                        ErrorOccur = input("Nan appears")
                        
    return net_local, running_loss_local, acc_local      

In [11]:
net_diff = Net()
net_diff.to(device)   # Storing network update
train_acc_history = np.zeros(epoch_max)
update_rho_history = np.zeros(epoch_max)

#inputs, labels = CU_x.to(device), torch.tensor(CU_y, dtype=torch.long).to(device)
#net, _, _ = model_localupdate(inputs, labels, net, init_E, batch_local, samples_CU, lr)

for epoch in range(epoch_max):  # loop over the dataset multiple times    
    ##################### Wireless Channel ###########################
    H_pow = np.ones((dev_num, B*N))    # Channel power gains, dev_num x BN
    
    ########## Update at the central unit #########
    with torch.no_grad():   # Reset update
        for param_diff in net_diff.parameters():
            param_diff = torch.zeros_like(param_diff)
    
    inputs, labels = CU_x.to(device), torch.tensor(CU_y, dtype=torch.long).to(device)
    net_CU, running_loss_CU, acc_CU = model_localupdate(inputs, labels, net, E, batch_local, samples_CU, lr)
    
    idx_s = 0
    sum_update_pow = 0.0
    with torch.no_grad():
        for param_diff, param_org, param_CU in zip(net_diff.parameters(), net.parameters(), net_CU.parameters()):
            param_diff = param_CU-param_org
            param_org += param_diff*(samples_CU/samples_global)
            sum_update_pow += torch.sum(param_diff*param_diff).item()
            idx_s += param_org.numel()
       
    rho = sum_update_pow/D
    print('avg_update_SNR: %f' % (rho/noise_pow))
            
    total = samples_CU*E
    print('[epoch: %d, CU] loss: %.3f  accuracy: %.3f' % (epoch + 1, running_loss_CU, acc_CU/total))
   
    
    ########## Update at distributed devices #########
    running_loss = 0.0
    acc_tmp = 0.0
    total = 0
    Y = np.zeros(B*N)
    
    for i in range(dev_num):   # get local dataset of i-th device; data is a list of [inputs, labels]
        inputs, labels = x[idx_localDB[i]].to(device), torch.tensor(y[idx_localDB[i]], dtype=torch.long).to(device)
        net_local, running_loss_local, acc_local = model_localupdate(inputs, labels, net, E, batch_local, samples_local[i], lr)
        running_loss += running_loss_local
        acc_tmp += acc_local
        
        with torch.no_grad():   
            update_stack = np.array([])
            delta_bar = np.zeros(B*N)
            for param_org, param_local in zip(net.parameters(), net_local.parameters()):
                update_stack = np.concatenate((update_stack, (param_local-param_org).view(-1).to("cpu").numpy()), 0)            
            delta_bar[:D] = update_stack*1.0
            delta_bar = delta_bar[permute_idx]            
            
            P_alloc = power_allocation(gamma, noise_pow, P, N, B, delta_bar, H_pow[i], rho, samples_local[i], samples_global)
                        
            X = np.sign(delta_bar)*np.sqrt(P_alloc)
            Y += np.sqrt(H_pow[i, :])*X            

        # print statistics
        total += E*samples_local[i]
        num_proc = int(dev_num/10)
        if i % num_proc == num_proc-1:
            print('[epoch: %d, device: %3d] loss: %.3f  accuracy: %.3f' % (epoch + 1, i + 1, running_loss/num_proc, acc_tmp/total))
            running_loss = 0.0
            acc_tmp = 0.0
            total = 0            
    
    tmp = np.sqrt(rho/gamma/noise_pow)*Y
    tmp = tmp[permute_idx_inv]
    delta_hat = torch.tensor(tmp).to(device)
    idx_s = 0
    with torch.no_grad():
        for param_org in net.parameters():   # Update model at the main server
            tmp_param = delta_hat[idx_s:idx_s+param_org.numel()]
            param_org += tmp_param.view(param_org.size())
            idx_s += param_org.numel()
            
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images_tmp, labels = data
            images = images_tmp.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted.cpu() == labels).sum().item()
    
    update_rho_history[epoch] = rho
    train_acc_history[epoch] = correct / total
    print('Accuracy for 10,000 test images: %.2f %%' % (100.0 * train_acc_history[epoch]))
    if correct / total > 0.99: break
    
print('Finished Training')

avg_update_SNR: 0.500702
[epoch: 1, CU] loss: 1.869  accuracy: 0.420
[epoch: 1, device:  16] loss: 1.045  accuracy: 0.631
[epoch: 1, device:  32] loss: 1.086  accuracy: 0.620
[epoch: 1, device:  48] loss: 1.042  accuracy: 0.647
[epoch: 1, device:  64] loss: 1.186  accuracy: 0.598
[epoch: 1, device:  80] loss: 1.156  accuracy: 0.607
[epoch: 1, device:  96] loss: 1.088  accuracy: 0.634
[epoch: 1, device: 112] loss: 1.070  accuracy: 0.632
[epoch: 1, device: 128] loss: 1.158  accuracy: 0.608
[epoch: 1, device: 144] loss: 1.130  accuracy: 0.612
[epoch: 1, device: 160] loss: 1.043  accuracy: 0.641
Accuracy for 10,000 test images: 68.81 %
avg_update_SNR: 0.444692
[epoch: 2, CU] loss: 1.069  accuracy: 0.690
[epoch: 2, device:  16] loss: 0.454  accuracy: 0.872
[epoch: 2, device:  32] loss: 0.472  accuracy: 0.873
[epoch: 2, device:  48] loss: 0.418  accuracy: 0.891
[epoch: 2, device:  64] loss: 0.482  accuracy: 0.864
[epoch: 2, device:  80] loss: 0.496  accuracy: 0.864
[epoch: 2, device:  96] lo

[epoch: 13, device: 144] loss: 0.167  accuracy: 0.948
[epoch: 13, device: 160] loss: 0.164  accuracy: 0.948
Accuracy for 10,000 test images: 93.65 %
avg_update_SNR: 0.090568
[epoch: 14, CU] loss: 0.353  accuracy: 0.915
[epoch: 14, device:  16] loss: 0.181  accuracy: 0.946
[epoch: 14, device:  32] loss: 0.169  accuracy: 0.948
[epoch: 14, device:  48] loss: 0.132  accuracy: 0.963
[epoch: 14, device:  64] loss: 0.170  accuracy: 0.950
[epoch: 14, device:  80] loss: 0.174  accuracy: 0.944
[epoch: 14, device:  96] loss: 0.151  accuracy: 0.955
[epoch: 14, device: 112] loss: 0.157  accuracy: 0.951
[epoch: 14, device: 128] loss: 0.215  accuracy: 0.939
[epoch: 14, device: 144] loss: 0.161  accuracy: 0.949
[epoch: 14, device: 160] loss: 0.158  accuracy: 0.950
Accuracy for 10,000 test images: 93.93 %
avg_update_SNR: 0.088996
[epoch: 15, CU] loss: 0.344  accuracy: 0.915
[epoch: 15, device:  16] loss: 0.175  accuracy: 0.947
[epoch: 15, device:  32] loss: 0.162  accuracy: 0.950
[epoch: 15, device:  4

[epoch: 26, device:  64] loss: 0.100  accuracy: 0.971
[epoch: 26, device:  80] loss: 0.123  accuracy: 0.960
[epoch: 26, device:  96] loss: 0.115  accuracy: 0.966
[epoch: 26, device: 112] loss: 0.116  accuracy: 0.963
[epoch: 26, device: 128] loss: 0.135  accuracy: 0.963
[epoch: 26, device: 144] loss: 0.123  accuracy: 0.960
[epoch: 26, device: 160] loss: 0.110  accuracy: 0.965
Accuracy for 10,000 test images: 95.95 %
avg_update_SNR: 0.095953
[epoch: 27, CU] loss: 0.300  accuracy: 0.920
[epoch: 27, device:  16] loss: 0.129  accuracy: 0.961
[epoch: 27, device:  32] loss: 0.120  accuracy: 0.964
[epoch: 27, device:  48] loss: 0.089  accuracy: 0.974
[epoch: 27, device:  64] loss: 0.097  accuracy: 0.970
[epoch: 27, device:  80] loss: 0.120  accuracy: 0.962
[epoch: 27, device:  96] loss: 0.113  accuracy: 0.966
[epoch: 27, device: 112] loss: 0.114  accuracy: 0.964
[epoch: 27, device: 128] loss: 0.131  accuracy: 0.963
[epoch: 27, device: 144] loss: 0.121  accuracy: 0.960
[epoch: 27, device: 160] 

avg_update_SNR: 0.106131
[epoch: 39, CU] loss: 0.284  accuracy: 0.925
[epoch: 39, device:  16] loss: 0.103  accuracy: 0.969
[epoch: 39, device:  32] loss: 0.082  accuracy: 0.975
[epoch: 39, device:  48] loss: 0.080  accuracy: 0.978
[epoch: 39, device:  64] loss: 0.078  accuracy: 0.975
[epoch: 39, device:  80] loss: 0.093  accuracy: 0.972
[epoch: 39, device:  96] loss: 0.092  accuracy: 0.972
[epoch: 39, device: 112] loss: 0.088  accuracy: 0.974
[epoch: 39, device: 128] loss: 0.092  accuracy: 0.974
[epoch: 39, device: 144] loss: 0.100  accuracy: 0.969
[epoch: 39, device: 160] loss: 0.087  accuracy: 0.973
Accuracy for 10,000 test images: 97.03 %
avg_update_SNR: 0.106401
[epoch: 40, CU] loss: 0.282  accuracy: 0.925
[epoch: 40, device:  16] loss: 0.101  accuracy: 0.970
[epoch: 40, device:  32] loss: 0.080  accuracy: 0.976
[epoch: 40, device:  48] loss: 0.082  accuracy: 0.979
[epoch: 40, device:  64] loss: 0.076  accuracy: 0.976
[epoch: 40, device:  80] loss: 0.091  accuracy: 0.972
[epoch: 4

[epoch: 51, device: 112] loss: 0.071  accuracy: 0.980
[epoch: 51, device: 128] loss: 0.071  accuracy: 0.981
[epoch: 51, device: 144] loss: 0.077  accuracy: 0.979
[epoch: 51, device: 160] loss: 0.078  accuracy: 0.975
Accuracy for 10,000 test images: 97.55 %
avg_update_SNR: 0.109565
[epoch: 52, CU] loss: 0.259  accuracy: 0.930
[epoch: 52, device:  16] loss: 0.087  accuracy: 0.975
[epoch: 52, device:  32] loss: 0.063  accuracy: 0.982
[epoch: 52, device:  48] loss: 0.057  accuracy: 0.984
[epoch: 52, device:  64] loss: 0.066  accuracy: 0.983
[epoch: 52, device:  80] loss: 0.077  accuracy: 0.976
[epoch: 52, device:  96] loss: 0.076  accuracy: 0.978
[epoch: 52, device: 112] loss: 0.070  accuracy: 0.980
[epoch: 52, device: 128] loss: 0.070  accuracy: 0.981
[epoch: 52, device: 144] loss: 0.076  accuracy: 0.980
[epoch: 52, device: 160] loss: 0.077  accuracy: 0.976
Accuracy for 10,000 test images: 97.60 %
avg_update_SNR: 0.109837
[epoch: 53, CU] loss: 0.257  accuracy: 0.930
[epoch: 53, device:  1

...

avg_update_SNR: 0.028774
[epoch: 251, CU] loss: 0.016  accuracy: 1.000
[epoch: 251, device:  16] loss: 0.012  accuracy: 0.998
[epoch: 251, device:  32] loss: 0.015  accuracy: 0.999
[epoch: 251, device:  48] loss: 0.013  accuracy: 0.998
[epoch: 251, device:  64] loss: 0.011  accuracy: 0.999
[epoch: 251, device:  80] loss: 0.012  accuracy: 0.999
[epoch: 251, device:  96] loss: 0.012  accuracy: 0.998
[epoch: 251, device: 112] loss: 0.011  accuracy: 0.999
[epoch: 251, device: 128] loss: 0.011  accuracy: 0.999
[epoch: 251, device: 144] loss: 0.013  accuracy: 0.999
[epoch: 251, device: 160] loss: 0.015  accuracy: 0.997
Accuracy for 10,000 test images: 98.66 %
avg_update_SNR: 0.028471
[epoch: 252, CU] loss: 0.016  accuracy: 1.000
[epoch: 252, device:  16] loss: 0.012  accuracy: 0.998
[epoch: 252, device:  32] loss: 0.015  accuracy: 0.999
[epoch: 252, device:  48] loss: 0.013  accuracy: 0.998
[epoch: 252, device:  64] loss: 0.011  accuracy: 0.999
[epoch: 252, device:  80] loss: 0.012  accuracy

[epoch: 263, device:  80] loss: 0.011  accuracy: 0.999
[epoch: 263, device:  96] loss: 0.012  accuracy: 0.998
[epoch: 263, device: 112] loss: 0.010  accuracy: 1.000
[epoch: 263, device: 128] loss: 0.010  accuracy: 1.000
[epoch: 263, device: 144] loss: 0.012  accuracy: 0.999
[epoch: 263, device: 160] loss: 0.014  accuracy: 0.997
Accuracy for 10,000 test images: 98.68 %
avg_update_SNR: 0.024173
[epoch: 264, CU] loss: 0.014  accuracy: 1.000
[epoch: 264, device:  16] loss: 0.012  accuracy: 0.998
[epoch: 264, device:  32] loss: 0.015  accuracy: 0.999
[epoch: 264, device:  48] loss: 0.012  accuracy: 0.999
[epoch: 264, device:  64] loss: 0.011  accuracy: 0.999
[epoch: 264, device:  80] loss: 0.011  accuracy: 0.999
[epoch: 264, device:  96] loss: 0.011  accuracy: 0.998
[epoch: 264, device: 112] loss: 0.010  accuracy: 1.000
[epoch: 264, device: 128] loss: 0.010  accuracy: 1.000
[epoch: 264, device: 144] loss: 0.012  accuracy: 0.999
[epoch: 264, device: 160] loss: 0.014  accuracy: 0.997
Accuracy

[epoch: 275, device: 160] loss: 0.013  accuracy: 0.998
Accuracy for 10,000 test images: 98.67 %
avg_update_SNR: 0.020408
[epoch: 276, CU] loss: 0.013  accuracy: 1.000
[epoch: 276, device:  16] loss: 0.011  accuracy: 0.998
[epoch: 276, device:  32] loss: 0.014  accuracy: 0.999
[epoch: 276, device:  48] loss: 0.011  accuracy: 0.999
[epoch: 276, device:  64] loss: 0.010  accuracy: 0.999
[epoch: 276, device:  80] loss: 0.010  accuracy: 0.999
[epoch: 276, device:  96] loss: 0.011  accuracy: 0.999
[epoch: 276, device: 112] loss: 0.009  accuracy: 1.000
[epoch: 276, device: 128] loss: 0.009  accuracy: 1.000
[epoch: 276, device: 144] loss: 0.011  accuracy: 0.999
[epoch: 276, device: 160] loss: 0.013  accuracy: 0.998
Accuracy for 10,000 test images: 98.67 %
avg_update_SNR: 0.020130
[epoch: 277, CU] loss: 0.013  accuracy: 1.000
[epoch: 277, device:  16] loss: 0.011  accuracy: 0.998
[epoch: 277, device:  32] loss: 0.014  accuracy: 0.999
[epoch: 277, device:  48] loss: 0.011  accuracy: 0.999
[epoch

[epoch: 288, device:  48] loss: 0.010  accuracy: 0.999
[epoch: 288, device:  64] loss: 0.010  accuracy: 0.999
[epoch: 288, device:  80] loss: 0.010  accuracy: 0.999
[epoch: 288, device:  96] loss: 0.010  accuracy: 0.999
[epoch: 288, device: 112] loss: 0.009  accuracy: 1.000
[epoch: 288, device: 128] loss: 0.009  accuracy: 1.000
[epoch: 288, device: 144] loss: 0.011  accuracy: 0.999
[epoch: 288, device: 160] loss: 0.012  accuracy: 0.998
Accuracy for 10,000 test images: 98.68 %
avg_update_SNR: 0.016925
[epoch: 289, CU] loss: 0.011  accuracy: 1.000
[epoch: 289, device:  16] loss: 0.010  accuracy: 0.999
[epoch: 289, device:  32] loss: 0.013  accuracy: 0.999
[epoch: 289, device:  48] loss: 0.010  accuracy: 0.999
[epoch: 289, device:  64] loss: 0.010  accuracy: 0.999
[epoch: 289, device:  80] loss: 0.010  accuracy: 0.999
[epoch: 289, device:  96] loss: 0.010  accuracy: 0.999
[epoch: 289, device: 112] loss: 0.009  accuracy: 1.000
[epoch: 289, device: 128] loss: 0.009  accuracy: 1.000
[epoch: 

[epoch: 300, device: 128] loss: 0.009  accuracy: 1.000
[epoch: 300, device: 144] loss: 0.010  accuracy: 0.999
[epoch: 300, device: 160] loss: 0.011  accuracy: 0.998
Accuracy for 10,000 test images: 98.70 %
Finished Training


In [12]:
from datetime import date
Date = '['+str(date.today())+']'
PATH_SIM = './results/Simulation_Env/'+Date+'SimEnv_Orth_Acc'+str(correct/total)+'Dmax'+str(cell_radius)+'Dens%.4f'%(density)+'CU'+str(samples_CU)   # save simulation environment
PATH_NET = './results/Network/'+Date+'Net_Orth_Acc'+str(correct/total)+'Dmax'+str(cell_radius)+'Dens%.4f'%(density)+'CU'+str(samples_CU)+'.pth'   # save neural network
PATH_ACC = './results/Accuracy/'+Date+'Acc_Orth_Acc'+str(correct/total)+'Dmax'+str(cell_radius)+'Dens%.4f'%(density)+'CU'+str(samples_CU)   # save accuracy history
PATH_RHO = './results/RHO/'+Date+'RHO_Orth_Acc'+str(correct/total)+'Dmax'+str(cell_radius)+'Dens%.4f'%(density)+'CU'+str(samples_CU)   # save SNR history

np.save(PATH_ACC, train_acc_history)
print(PATH_ACC+'.npy is saved')
np.savez(PATH_SIM, lr=lr, batch_local=batch_local, E=E, epoch_max=epoch_max, D=D, noise_pow_dBm=noise_pow_dBm, P_dBm=P_dBm, gamma=gamma, B=B, N=N, cell_radius=cell_radius, density=density, dev_num=dev_num, pathloss_exponent=pathloss_exponent, num_classes=num_classes, samples_avg_local=samples_avg_local, samples_local=samples_local, samples_CU=samples_CU)
print(PATH_SIM+'.npz is saved')
np.save(PATH_RHO, update_rho_history)
print(PATH_RHO+'.npy is saved')
torch.save(net.state_dict(), PATH_NET)
print(PATH_NET+'.pth is saved')



./results/Accuracy/[2021-09-09]Acc_Orth_Acc0.987Dmax400Dens0.0003CU200.npy is saved
./results/Simulation_Env/[2021-09-09]SimEnv_Orth_Acc0.987Dmax400Dens0.0003CU200.npz is saved
./results/RHO/[2021-09-09]RHO_Orth_Acc0.987Dmax400Dens0.0003CU200.npy is saved
./results/Network/[2021-09-09]Net_Orth_Acc0.987Dmax400Dens0.0003CU200.pth.pth is saved


In [ ]:
train_acc_history = np.load(PATH_ACC+'.npy')
plt.plot(np.arange(epoch_max), train_acc_history)
plt.xlabel('Epoch')
plt.ylabel('Test Accuracy')
plt.show()

update_rho_history = np.load(PATH_RHO+'.npy')
plt.plot(np.arange(epoch_max), 10*np.log10(update_rho_history/noise_pow))
plt.xlabel('Epoch')
plt.ylabel('Update SNR [dB]')
plt.show()

In [ ]:
net = Net()
net.load_state_dict(torch.load(PATH_NET))
outputs = net(images.cpu())

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %.2f %%' % (100.0 * correct / total))

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels)
        for i in range(labels.size(0)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(10):
    print('Accuracy of %d : %.2f %%' % (classes[i], 100 * class_correct[i] / class_total[i]))

In [ ]:
print('Number of total devices: ', dev_num)
print('Distance of device %d: '%(dev_num-1), dev_dist[dev_num-1])
print('Probability of the utilized sub-channels: ', np.sum(P_alloc>0)/D)
print('Remaining powers of device %d N symbols : '%(dev_num-1), P-np.sum(P_alloc)/N)

In [ ]:
tmp = list(net_diff.parameters())

params = np.array([])
for i in range(len(tmp)):
    params = np.append(params, tmp[i].cpu().detach().numpy().reshape(-1))
    
print(params.shape)

max_param = np.max(params*params)
min_param = np.min(params*params)
bins = np.linspace(min_param, max_param, num=100)

_ =plt.hist(params*params, bins=bins)


tmp_idx = np.random.permutation(params.shape[0])
samples_idx = tmp_idx[0:1024]
samples = params[samples_idx]

In [ ]:
tmp_idx = np.random.permutation(params.shape[0])
samples_idx = tmp_idx[0:1024]
#samples_idx = np.arange(0,1024,1)
samples = params[samples_idx]
pows = samples*samples

max_param = np.max(pows)
min_param = np.min(pows)
bins = np.linspace(min_param, max_param, num=100)

count = plt.hist(pows, bins=bins)

mean_pow = np.mean(pows)
y=count[0][0]*np.exp(-bins/mean_pow)
plt.plot(bins, y)

print(mean_pow)

In [ ]:
np.sum((X**2).reshape(N, B), axis=1)